# Importing libriries : 

In [2]:
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet50
from collections import deque

c:\Users\ashraf\anaconda3\envs\testenvironment\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Filtring the data : 

In [4]:
Sport_Labels = set(['boxing', 'swimming', 'table_tennis'])
print("Images are being loaded ... ")   
datapath = '../data'
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for image_path in pathToImages:
    label = image_path.split(os.path.sep)[-2]
    if label not in Sport_Labels:
        continue
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Corrected the function call
    image = cv2.resize(image, (244, 244))
    data.append(image)
    labels.append(label)

Images are being loaded ... 


In [5]:
data = np.array(data)
labels = np.array(labels)

In [6]:
lb = label_binarize(labels, classes=list(Sport_Labels))
class_labels = list(Sport_Labels)  # Store the class labels separately
labels = torch.tensor(lb, dtype=torch.float32)  # Convert the labels to float

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [8]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123.68/255.0, 116.779/255.0, 103.939/255.0], std=[1, 1, 1]),
])

In [10]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123.68/255.0, 116.779/255.0, 103.939/255.0], std=[1, 1, 1]),
])

In [11]:
train_dataset = CustomDataset(X_train, Y_train, transform=train_transform)
test_dataset = CustomDataset(X_test, Y_test, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.base_model = resnet50(pretrained=True)
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()
        self.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.base_model(x)
        out = self.fc(features)
        return out

In [13]:
model = ResNetModel(num_classes=len(Sport_Labels))
for param in model.base_model.parameters():
    param.requires_grad = False

# Optimizer and Loss Function
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [14]:
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, torch.argmax(labels, dim=1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(train_loader)}")

In [72]:
num_epochs = 1
train(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch [1/1] Loss: 0.38284807473421095


In [15]:
torch.save(model.state_dict(), 'outputmodel.pth')

In [16]:
import pickle
with open('label_binarizer.pkl', 'wb') as f:
    pickle.dump(lb, f)

In [17]:
# Loading the model and label binarizer
model = ResNetModel(num_classes=len(Sport_Labels))
model.load_state_dict(torch.load('outputmodel.pth'))
model.eval()

ResNetModel(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [18]:
with open('label_binarizer.pkl', 'rb') as f:
    lb = pickle.load(f)

In [29]:
capture_video = cv2.VideoCapture('./How to swim.mp4')
outputvideo = 'output_video.avi'
writer = None
(Width, Height) = (None, None)
mean = torch.tensor([123.68/255.0, 116.779/255.0, 103.939/255.0]).unsqueeze(0).unsqueeze(2).unsqueeze(3)

queue = deque(maxlen=128)

In [30]:
font = cv2.FONT_HERSHEY_SIMPLEX
font_size = 1.25
font_thickness = 2
text_color = (255, 255, 255)  # White color
text_background_color = (0, 0, 0)  # Black background color
text_opacity = 0.7  # Adjust the opacity of the text

while True:
    (taken, frame) = capture_video.read()
    if not taken:
        break
    if Width is None or Height is None:
        (Height, Width) = frame.shape[:2]

    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (244, 244)).astype("float32") / 255.0
    frame = torch.tensor(frame.transpose(2, 0, 1)).unsqueeze(0)
    frame -= mean
    with torch.no_grad():
        frame = model(frame)
    queue.append(frame.cpu().numpy())
    results = np.array(queue).mean(axis=0)
    i = np.argmax(results)
    label = class_labels[i]  # Use the stored class labels for inference
    text = f"Sport: {label}"

    # Draw background rectangle
    cv2.rectangle(output, (30, 40), (400, 80), text_background_color, -1)

    # Draw text on the video
    cv2.putText(output, text, (45, 70), font, font_size, text_color, font_thickness, cv2.LINE_AA)

    # Make the text blend with the frame
    cv2.addWeighted(output, text_opacity, output, 1 - text_opacity, 0, output)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outputvideo, fourcc, 30, (Width, Height), True)
    writer.write(output)
    cv2.imshow("In progress", output)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

KeyboardInterrupt: 

In [31]:
print("Finalizing...")
writer.release()
capture_video.release()
cv2.destroyAllWindows()

Finalizing...
